## Submission Requirement # 5

### CNN for image recognition with MNIST dataset:

In [1]:
# Import the required libraries

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

In [2]:
# import dataset from the api
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
type(mnist)

tensorflow.contrib.learn.python.learn.datasets.base.Datasets

In [5]:
# To check how many images we get for training and testing
mnist.train.num_examples

55000

In [6]:
mnist.test.num_examples

10000

### Preparation for Building CNN Model : Define Supporting Functions

#### Initialize Weights in Filter

In [7]:
# This function returns a tf.Variable used to store weights in a filter
# This variable is initialized with values that can be used to initialize weights
# The values are random numbers

def initialize_weights(filter_shape):
    init_random_dist =tf.truncated_normal(filter_shape, stddev=0.1)
    return (tf.Variable(init_random_dist) )

#### Initialize Bias

In [8]:
# This function returns a tf.Variable used to store bias
# This variable is initialized with values that can be used to initialize bias
# The value is initialized to 0.1

def initialize_bias(bias_shape):
    init_bias_vals =tf.constant(0.1, shape=bias_shape)
    return (tf.Variable(init_bias_vals) )

#### Setup Convolutional Layer and Perform Convolution Computation : Dot Product (x * W)

In [9]:
# Define a function to setup a convolutional layer (conv 2d)
# Parameters:
# ..) Inputs : [batch, H, W, Channels]
#..) filter_shape:
#[filter H, filter W, in_channels(in_depth=in_num_filters),out_channels(out_depth=out_num_filters)]
# for example : [5,5,1,32]
# Return : Outputs of the layer:The dot_product(inputs, filter_shape):

def create_convolution_layer_and_compute_dot_product(inputs,filter_shape):
    filter_initialized_with_weights= initialize_weights(filter_shape)
    conv_layer_outputs=tf.nn.conv2d(inputs,filter_initialized_with_weights,strides=[1,1,1,1],padding='SAME')
    return(conv_layer_outputs)
    

#### Setup a ReLU Layer and Perform Computation : Dot Product + Bias (x .W+b)

In [10]:
# Define a function
#..) First,  to setup a reLU layer: an activation function layer
#..) Then, perform the computation : dot product +bias (x * w) +b
# ReLU : Rectified Linear Unit - a popular activation function in CNN
# In this layer, the bias has been initialized

# Parameters:
# ..) Inputs : Outputs from the preceding convolution layer : Dot product inputs*weights
# ..) filter_shape
#[filter H, filter W, in_channels(in_depth=in_num_filters),out_channels(out_depth=out_num_filters)]
# for example : [5,5,1,32]
# Return : Outputs of the layer: dot product +b : (x *W)+ b


def create_relu_layer_and_compute_dot_product_plus_b(inputs,filter_shape):
    b=initialize_bias( [filter_shape[3] ])
    relu_layer_outputs=tf.nn.relu(inputs + b)
    return(relu_layer_outputs)

#### Setup a Pooling Layer and Reduce Spatial Size

In [11]:
# Define a function
#..) First,  to create a pooling layer
#..) Then, to reduce the spatial size of the inputs data

# Pooling Method : Max Pooling
# Kernel size : 2 X 2
# Stride : 2

# Parameters:
# ..) Inputs : Outputs from the preceding layer 
# ..) Return : Outputs of the layer

def create_maxpool2by2_and_reduce_spatial_size(inputs):
    pooling_layer_outputs = tf.nn.max_pool(inputs, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    return(pooling_layer_outputs)
                                           
                                           
                                           
                                           

#### Setup Fully Connected Layer and Perform Computation:(inputs * Weights) +Bias

In [12]:
# Define a function to set up a fully connected layer
# Also perform computation: (Dot Product (inputs*weights))+ bias
# Return the results
# Parameters:
#..) inputs:  Outputs of the preceding layer
#..) size: The size of the outputs, t.e. number of out_channels

def create_fully_connected_layer_and_compute_dotproduct_plus_bias (inputs, output_size):
    input_size = int(inputs.get_shape() [1] )
    W = initialize_weights([input_size, output_size ] )
    b = initialize_bias ([output_size])
    fc_xW_plus_bias_outputs = tf.matmul(inputs, W) + b
    return(fc_xW_plus_bias_outputs)
    

### Phase 1 : Build the Convolutional Neural Network 

#### Create placeholders for inputs and labels : x & y_true

In [13]:
# PLACEHOLDER

# Create a placeholder for the inputs data : x
# x : a 2D array
# x: a placeholder that can hold any number of rows/record
# ..) Each row/record is a vector ( 1D array ) to hold data for one image
# ..) Each row/record/image has 784 values /elements : 1 pixel =  value (28 *28 =784 )

x = tf.placeholder (tf.float32, shape = [None, 784])

In [14]:
# PLACEHOLDER

# Create a placeholder for the labels of the inputs data  : y_true
# y_true : a 2D array
# y_true : Can hold any number of rows/records
# Each row/record is a vector (1D-array) of 10 values that indicate a digit between 0 ….9
# ..) Each row is the label of one image , i.e. one digit
# —) Each row/record : The label is stored in the on-hot format
# ..) One-hot format :  [ 1 0 0 0 0 0 0 0 0 0 0 ] —> 0 ; [ 0 0 0 0 0 0 0 0 1 0]—>8

y_true= tf.placeholder (tf.float32, shape = [None, 10 ])

In [16]:
#### Reshape the Input placeholder X
# Prepare feeding inputs into the conv layer
# Reshape the input x: A placeholder
# From 1D array (vector)—> Original input shape : 4D :[Batch, H, W, depth channels ]
# Depth = color channels : gray scale =1
# Reshaped inputs : x_image [1,28,28,1]

x_image=tf.reshape(x,[-1,28,28,1])

#### Create the Convolutional layer, ReLU layer, and perform computation : x * w+ b(only one layer)

In [17]:
# Create the convolution layer (only one layer)
# Inputs: x_image, reshape inputs with shape [1,28,28,1]
# filter _shape : [5,5,1,32]
# ..) Filter : 5* 5
# ..) Input channels : 1
# ..) Output channels : 32

# Create the 1st convolution layer
# Then learn/extract the features, get the results (outputs) : Dot product of inputs*weights
# Return the outputs of the layer

conv_layer_1_outputs \
  =create_convolution_layer_and_compute_dot_product(x_image,filter_shape=[5,5,1,32])

#### Create the ReLU layer for the convolution layer

In [18]:
# Accept the outputs from the conv layer as the inputs
# Perform the computation at the layer : add inputs+ bias
# Return the outputs of the layer

conv_relu_layer_1_outputs \
  =create_relu_layer_and_compute_dot_product_plus_b(conv_layer_1_outputs,filter_shape=[5,5,1,32])


#### Create the Pooling Layer and reduce spatial size

In [19]:
# Create the 1st pooling layer
# Then reduce the spatial size of the input data
# Return : Outputs of the layer
pooling_layer_1_outputs=create_maxpool2by2_and_reduce_spatial_size(conv_relu_layer_1_outputs)

#### Reshape/Flatten Data Making it Ready to be Def into 1st FC layer

In [20]:
# Reshape and flatten the output of the 2nd pooling layer
# Prepare to feed the output data into the 1st fully connected layer

pooling_layer_1_outputs_flat=tf.reshape(pooling_layer_1_outputs,[-1, 14*14*32])

#### Create 1st FC Layer, ReLU Layer, and output Data to Dropout Layer

In [21]:
# First create  the FC (Fully Connected ) layer
# Feed the output(already flattened ) of the 2nd pooling layer as the inputs into this layer
# Then perform the computation : dot product _bias ( x *W) + b

# Parameters:
# ..) pooling_layer_2_outputs_flat
# ..) output_size: 1024 out channels

# Return: Outputs of the computation (x*W)_b

fc_layer_1_outputs \
   = create_fully_connected_layer_and_compute_dotproduct_plus_bias (pooling_layer_1_outputs_flat,output_size =1024)

fc_relu_layer_1_outputs=tf.nn.relu(fc_layer_1_outputs)


#### Create Dropout Layer and Dropout of Outputs Randomly

In [22]:
# Declare a placeholder to hold the value of probability (percentage x) to keep:
# i.e. the % of total output channels that will be kept
# e.g. keep_prob=hold_prob= 50% —> keep 50 %..> Dropout 50%
# which nodes/channels to keep or dropout : selected randomly

hold_prob= tf.placeholder (tf.float32)

In [23]:
# Dropout
# set the outputs to 0 (zero) so that they will be ignored in the next layer

fc_dropouts_outputs=tf.nn.dropout(fc_relu_layer_1_outputs, keep_prob=hold_prob)

#### Create Final FC Layer, Compute (x.W_b) and produce final outputs

In [24]:
# Create the final FC layer
# Then compute: x*W_b
# parameters:
# ..) fc_dropouts_outputs : Outputs from the dropout  layer
# ..)

# Return y_pred: final predicted outputs, I.e., final classification outputs

y_pred = create_fully_connected_layer_and_compute_dotproduct_plus_bias(fc_dropouts_outputs,output_size=10)

#### Define Loss Function and Calculate Softmax Cross Entropy Loss

In [25]:
# Define loss function :cross-entropy with logins . i.e. with the final outputs
# Calculate the softmax entropy loss


softmax_cross_entropy_loss=tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [26]:
# Compute the means of losses

cross_entropy_mean = tf.reduce_mean(softmax_cross_entropy_loss)

#### Create an Optimizer to Optimize CNN Model and set learning rate

In [27]:
# Get an ADAM optimizer

optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

#### Create a Trainer to Train CNN Model

In [28]:
# Create a CNN model trainer that can train the model
# And optimizer the model by minimizing the softmax cross_entropy loss

cnn_trainer = optimizer.minimize (cross_entropy_mean)

### Train and Test CNN Deep Learning Model on MNIST Dataset

#### Create a variable initializer to initialize All Variables

In [29]:
# Get a variable initializer

vars_initializer = tf.global_variables_initializer()

In [30]:
# x: mnist.train: 50000 images
# Each time of training (run the whole process ) = 1 step
# Each step, i.e., each time of training : Use one batch of inputs
# batch size = 50 images —> Total number of batches : 50000/50 = 1000 batches
# steps = 3000: Finish the training after running 3000 times

steps = 3000


#### Run tf.Session () to Train and test  Deep Learning CNN Model

In [31]:
with tf. Session () as sess:
    
    
    # First, run vars_initializer to initialize all variables
    sess.run(vars_initializer)
    
    for i in range (steps):
        
        
        # Each batch: 50 images
        batch_x,batch_y =mnist.train.next_batch(50)
        
        # Train the model cnn_trainer
        # Dropout keep_prob (% to keep) : 0.5 ..> 50% will be dropped out 
        
        sess.run(cnn_trainer, feed_dict={x:batch_x, y_true:batch_y, hold_prob :0.5})
        
        # Test the model: at each 100th step
        # Run this block of code for each 100 times of training, each time run a batch
        if i % 100 == 0:
            
            print('ON STEP: {}'.format(i) )
            print('ACCURACY: ')
            
            # Compare to find matches of y_pred and y_true
            matches =tf.equal( tf.argmax(y_pred, axis =1 ), tf.argmax(y_true,  axis =1) )
            
            
        
           # Cast the matches from integers to tf.float32
           # Calculate the accuracy using the mean of matches
        
            acc=tf.reduce_mean(tf.cast(matches,tf.float32) )
            
           # Test the model at each 100th step
           # Using the test dataset
           # Dropout: NONE because of test, not training
            
            test_accuracy=sess.run(acc, feed_dict = {x: mnist.test.images , \
                                                                 y_true: mnist.test.labels, \
                                                                 hold_prob: 1.0}  )
            print(test_accuracy)
            print('\n')
            
            
            
        

        
           
            

ON STEP: 0
ACCURACY: 
0.0998


ON STEP: 100
ACCURACY: 
0.9037


ON STEP: 200
ACCURACY: 
0.9384


ON STEP: 300
ACCURACY: 
0.944


ON STEP: 400
ACCURACY: 
0.9589


ON STEP: 500
ACCURACY: 
0.9668


ON STEP: 600
ACCURACY: 
0.9697


ON STEP: 700
ACCURACY: 
0.9746


ON STEP: 800
ACCURACY: 
0.9732


ON STEP: 900
ACCURACY: 
0.9721


ON STEP: 1000
ACCURACY: 
0.9775


ON STEP: 1100
ACCURACY: 
0.9782


ON STEP: 1200
ACCURACY: 
0.9776


ON STEP: 1300
ACCURACY: 
0.9785


ON STEP: 1400
ACCURACY: 
0.9826


ON STEP: 1500
ACCURACY: 
0.9822


ON STEP: 1600
ACCURACY: 
0.9828


ON STEP: 1700
ACCURACY: 
0.986


ON STEP: 1800
ACCURACY: 
0.9827


ON STEP: 1900
ACCURACY: 
0.9837


ON STEP: 2000
ACCURACY: 
0.983


ON STEP: 2100
ACCURACY: 
0.9834


ON STEP: 2200
ACCURACY: 
0.9832


ON STEP: 2300
ACCURACY: 
0.9852


ON STEP: 2400
ACCURACY: 
0.9843


ON STEP: 2500
ACCURACY: 
0.9843


ON STEP: 2600
ACCURACY: 
0.9842


ON STEP: 2700
ACCURACY: 
0.9853


ON STEP: 2800
ACCURACY: 
0.9856


ON STEP: 2900
ACCURACY: 
0.98